In [10]:
from tokenizers.models import BPE, WordPiece
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import Sequence, Lowercase
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import BpeTrainer
from transformers import GPT2Tokenizer, GPT2TokenizerFast, GPT2Config,TFGPT2LMHeadModel, TFGPT2Model
# from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import string
import time

In [14]:
data = pd.read_csv('data.csv')

# Combine relevant columns into a single text input
texts = data['brand'].astype(str) + ' ' + data['title'].astype(str) + ' ' + data['category'].astype(str) + ' ' + data['sold_price'].astype(str) + ' ' + data['actual_price'].astype(str)
url = data['url']

x = []
for text in texts:
    x.append(str(texts))

In [16]:
len(x)

63474

In [9]:
url

0        https://www.flipkart.com/kotty-regular-fit-wom...
1        https://www.flipkart.com/tyffyn-slim-women-bla...
2        https://www.flipkart.com/kotty-regular-fit-wom...
3        https://www.flipkart.com/mokshi-women-printed-...
4        https://www.flipkart.com/lee-tex-regular-fit-w...
                               ...                        
63469    https://www.flipkart.com/endues-printed-satin-...
63470    https://www.flipkart.com/not-full-sleeve-color...
63471    https://www.flipkart.com/atxp-top-pyjama-set-b...
63472    https://www.flipkart.com/fleximaa-full-sleeve-...
63473    https://www.flipkart.com/lavish-club-sleeveles...
Name: url, Length: 63474, dtype: object

In [18]:
MAX_LENGTH = np.max([len(str(y).split()) for y in x]) + 5
MAX_LENGTH

102

In [19]:
BOS_TOKEN = "<s>"
PAD_TOKEN = "<pad>"
EOS_TOKEN = "</s>"

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2", max_length=MAX_LENGTH)
tokenizer.add_special_tokens({
    "eos_token": EOS_TOKEN,
    "bos_token": BOS_TOKEN,
    "unk_token": "<unk>",
    "pad_token": PAD_TOKEN,
#     "mask_token": "<mask>"
})

4

In [20]:
train_data = [BOS_TOKEN + " " + str(ex) + " " + EOS_TOKEN for ex in x]

In [22]:
train_data_ = [tokenizer(str(y), return_tensors='tf', max_length=MAX_LENGTH, truncation=True, pad_to_max_length=True, add_special_tokens=True)['input_ids'] for y in train_data]
train_data_in = tf.squeeze(tf.convert_to_tensor(train_data_), axis=1)

/home/priyash7/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
2023-08-17 01:15:02.049365: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [23]:
train_data_in.shape

TensorShape([63474, 102])

In [25]:
urls = url.tolist()

# Convert labels to numerical indices
url_to_index = {url: i for i, url in enumerate(urls)}
labels = [url_to_index[url] for url in urls]
y_train_in = tf.constant(labels, dtype=tf.int32)

In [34]:
train_data_mask_ = [tokenizer(str(y), return_tensors='tf', max_length=MAX_LENGTH, truncation=True, pad_to_max_length=True, add_special_tokens=True)["attention_mask"] for y in train_data]
train_data_mask = tf.squeeze(tf.convert_to_tensor(train_data_mask_), axis=1)



<tf.Tensor: shape=(63474, 102), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)>

In [27]:
model = TFGPT2Model.from_pretrained("gpt2", use_cache=False,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id)
model.training = True
model.resize_token_embeddings(len(tokenizer))

All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [28]:
for layer in model.layers:
    layer.trainable = False
     

model.summary()

Model: "tfgpt2_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  124442880 
 er)                                                             
                                                                 
Total params: 124442880 (474.71 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 124442880 (474.71 MB)
_________________________________________________________________


In [29]:
input = tf.keras.layers.Input(shape=(None,), dtype='int32')
mask = tf.keras.layers.Input(shape=(None,), dtype='int32')
x = model(input, attention_mask=mask)
x = tf.reduce_mean(x.last_hidden_state, axis=1)
x = tf.keras.layers.Dense(16, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
     

clf = tf.keras.Model([input, mask], output)
     

clf.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 tfgpt2_model (TFGPT2Model)  TFBaseModelOutputWithPastA   1244428   ['input_1[0][0]',             
                             ndCrossAttentions(last_hid   80         'input_2[0][0]']             
                             den_state=(None, None, 768                                           
                             ),                                                               

In [30]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5,epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
base_learning_rate = 0.0005
optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
#loss=tf.keras.losses.BinaryCrossentropy()
loss=tf.keras.losses.BinaryCrossentropy()

clf.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
     

callbacks = tf.keras.callbacks.EarlyStopping(
        monitor="accuracy", verbose=1, patience=3, restore_best_weights=True)

In [32]:
tf.config.experimental_run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [33]:
history = clf.fit([train_data_in, train_data_mask], y_train_in, epochs=5, batch_size=32, validation_split=0.2, callbacks=callbacks)

Epoch 1/30


/home/priyash7/.local/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


  26/1587 [..............................] - ETA: 3:54:13 - loss: -217237.5312 - accuracy: 0.0000e+00

KeyboardInterrupt: 